In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pyctcdecode
!pip install transformers
!pip install kenlm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 25.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184423 sha256=c97ac3ec5b275a6c19bd09c018acc20b923cd3a8f48d7cfc0b297e9ce3e9e9f2
  Stored in directory: /root/.cache/pip/wheels/fd/80/e0/18f4148e863fb137bd87e21ee2bf423b81b3ed6989dab95135
Successfully built kenlm


# Packages

In [6]:
import pandas as pd
import numpy as np
import torch
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
import pyctcdecode
import kenlm
import csv
from transformers import pipeline
import random

DATASET

In [7]:
model_path = "bangla-speech-processing/BanglaASR"
DEFAULT_TRANSCRIPTION = "ও"
kenlm_model_path = "/content/drive/MyDrive/bengali-speech/lm_v9.binary/lm_v9.binary"
ALPHA = 0.5
BETA = 1.0

ASR MODEL

In [8]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mp3_path = "https://huggingface.co/bangla-speech-processing/BanglaASR/resolve/main/mp3/common_voice_bn_31515636.mp3"
model_path = "bangla-speech-processing/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)


speech_array, sampling_rate = torchaudio.load(mp3_path, format="mp3")
speech_array = speech_array[0].numpy()
speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features #file

predicted_ids = model.generate(inputs=input_features.to(device))[0]


transcription = processor.decode(predicted_ids, skip_special_tokens=True)

print(transcription)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `

এই প্রজাতিটির নানা ভেষজ গুণাগুণ আছে।


In [9]:
file_path = "/content/drive/MyDrive/bengali-speech/vocab-4500000_v9.txt.zip"

try:
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
        print(content)
except UnicodeDecodeError:
    with open(file_path, "r", encoding="latin-1") as file:
        content = file.read()
content = content.encode("utf-8", errors="replace").decode("utf-8")


In [10]:
model_path = "bangla-speech-processing/BanglaASR"
pipe = pipeline(task="automatic-speech-recognition", model=model_path)
directory_path = '/content/drive/MyDrive/bengali-speech/train_mp3s'

# Get a list of MP3 files  in the directory
mp3_files = [filename for filename in os.listdir(directory_path) if filename.endswith(".mp3")]
selected_file = random.choice(mp3_files)
mp3_file_path = os.path.join(directory_path, selected_file)
result = pipe(mp3_file_path)["text"]
csv_file_path = '/content/drive/MyDrive/bengali-speech/submission.csv'

# Check if the CSV file already exists
csv_file_exists = os.path.exists(csv_file_path)

# Write the transcription result to the CSV file
with open(csv_file_path, mode='a', newline='') as csvfile:
    fieldnames = ['File', 'Transcription']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write header if the file is newly created
    if not csv_file_exists:
        writer.writeheader()
        with open("output.txt", "w", encoding="utf-8") as file:
             file.write("your_string_containing_non_ascii_characters")




print(f"Transcription for {selected_file} has been saved to {csv_file_path}")
print(f"Transcription for {selected_file}: {result}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription for 42c0caa000dd.mp3 has been saved to /content/drive/MyDrive/bengali-speech/submission.csv
Transcription for 42c0caa000dd.mp3: পাড়ার নানা স্থানে খুজিয়া কুথাও পাইরোনা।
